<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# **Capstone Project** 
# **Real-Time Facial Emotion Recognition - Notebook 3**

## Problem Statement

To create a model that can **accurately distinguish emotions based on facial expressions**.

The models are evaluated based on the following criteria:

1. Accuracy scores (the higher, the better)
2. Delta between train and test scores (the smaller, the better)

## Structure & Workflow

To organise my work better, I have organised this project into two notebooks, across six parts: 

**Notebook 1:** 
* Part 1 : Data Import
* Part 2: Exploratory Data Analysis

**Notebook 2:**
* Part 3: Data Preprocessing
* Part 4: Baseline Model: A CNN model built from scratch
* Part 5: Transfer Learning & Model Evaluation

**Notebook 3:**
* Part 6: Model Deployment

In [1]:
# import required libraries/packages

import cv2
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from numpy import asarray

In [2]:
# as the VGGFace finetuned model scored the best, we will load it and use it for the real-time face recognition task

model = tf.keras.models.load_model("./saved_models/model4_vggface_finetuned_bestmodel.hdf5")

## 6. Model Deployment

In [4]:
class_to_label = {0: 'angry', 1: 'fear', 2: 'happy', 3: 'sad', 4: 'surprise'}
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# connect to webcame
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Unable to launch webcam")

# loop through every frame until we close the webcam    
while True:
    ret,frame = cap.read()
    
    #to draw out rectangle across the face
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.1,4)

    for face in faces:
        x, y, w, h = face
        x1 = x
        x2 = x+w
        y1 = y
        y2 = y+h

        ori_face = frame[y1:y2, x1:x2]
        image = Image.fromarray(ori_face)
        image = image.resize((48, 48))
        face_array = asarray(image)
        
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
        # use the selected model to predict emotion on each frame
        pred = model.predict(tf.constant(face_array.reshape(1, 48, 48, 3)))
        label = class_to_label[np.argmax(pred)]
        label_txt = "{}: {}%".format(label, np.round(max(pred[0])*100,1))
        
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(frame,label_txt,
               (x,y-10),
               font,0.8,
               (255,255,255),2)
        
    # show real-time video
    cv2.imshow('Original Video',frame)
    
    # checks whether q has been hit and stops the loop
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

# release the webcam
cap.release()
# close the frame
cv2.destroyAllWindows()

1/1 [==============================] - 0s 96ms/step
